# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [13]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [14]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [15]:
# Import the necessary CSVs to Pandas DataFrames
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_df = pd.read_csv(file_path, index_col="year")

file_path_2 = Path('Data/neighborhoods_coordinates.csv')
location_data = pd.read_csv(file_path_2)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [31]:
def neighborhood_map():
    """Neighborhood Map."""
    neighborhood_avg = sfo_df.groupby(['neighborhood']).mean().reset_index().rename(columns = {'neighborhood' : 'Neighborhood'})
    neighborhood_avg_2 = neighborhood_avg.reset_index().rename(columns = {'neighborhood' : 'Neighborhood'})
    location_data_2 = pd.merge(location_data, neighborhood_avg_2, on = 'Neighborhood')
    return px.scatter_mapbox(
        location_data_2,
         lat='Lat',
         lon='Lon',
         size='sale_price_sqr_foot',
         color='gross_rent',
         hover_name='Neighborhood',
         zoom=10)


In [45]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    housing_units = sfo_df['housing_units'].groupby('year').mean()
    return housing_units.hvplot.bar(
        x = 'year',
        y = 'housing_units',
        xlabel = 'Housing Units',
        ylabel = 'Year',
        title = 'Mean Housing Units Per Year',
        ylim = (370000, 385000) ).opts(yformatter = '%0f')


def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    avg_gross_rent = sfo_df[['sale_price_sqr_foot','gross_rent']].groupby('year').mean()
    return avg_gross_rent.hvplot.line(
    title='Average Monthly Rent',
    xlabel = 'Year',
    ylabel = 'Gross Rent')



def average_sales_price():
    """Average Sales Price Per Year."""
    sfo_avg_data = sfo_df[['sale_price_sqr_foot','gross_rent']].groupby('year').mean()
    avg_sales_price = sfo_avg_data['sale_price_sqr_foot']
    return avg_sales_price.hvplot.line(
    title = 'Avergae Sale Price Per Square Foot',
    xlabel = 'Year',
    ylabel = 'Average Sale Price')
    
   



def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    avg_neighborhood_price = sfo_df.groupby(['year', 'neighborhood']).mean().reset_index()
    return avg_neighborhood_price.hvplot.line(
    title = 'Average Price Per Sqr Foot',
    x ='year',
    y = 'sale_price_sqr_foot',
    groupby= 'neighborhood',
    line_color= 'darkmagenta')
    
    



def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    top_10_exspensive = sfo_df.groupby('neighborhood').mean().nlargest(10,'sale_price_sqr_foot')
    return top_10_exspensive.hvplot.bar(
        y='sale_price_sqr_foot',
        x = 'neighborhood',
        title = 'Most Expensive Neighborhoods',
        rot=90,
        ylim=(600, 950))

    


def most_expensive_neighborhoods_rent_sales():
    """Comparison of Rent and Sales Prices of Most Expensive Neighborhoods."""   
    most_expensive_rent_sales = sfo_df.groupby(['year', 'neighborhood']).mean().reset_index()
    top_10_plot = most_expensive_rent_sales[['sale_price_sqr_foot', 'gross_rent', 'neighborhood', 'year']] 
    return top_10_plot.hvplot.bar(
        x = 'year',
        groupby = 'neighborhood',
        rot = 90,
        title= 'SFOs 10 Most Expensive Neighborhoods',
        xlabel='') 



def neighborhood_map():
    """Neighborhood Map."""
    neighborhood_avg = sfo_df.groupby(['neighborhood']).mean().reset_index().rename(columns = {'neighborhood' : 'Neighborhood'})
    neighborhood_avg_2 = neighborhood_avg.reset_index().rename(columns = {'neighborhood' : 'Neighborhood'})
    location_data_2 = pd.merge(location_data, neighborhood_avg_2, on = 'Neighborhood')
    return px.scatter_mapbox(
        location_data_2,
         lat='Lat',
         lon='Lon',
         size='sale_price_sqr_foot',
         color='gross_rent',
         hover_name='Neighborhood',
         zoom=10)



## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [71]:
full_list = pn.Row(housing_units_per_year,
       average_gross_rent,
       average_sales_price,
       average_price_by_neighborhood,
       top_most_expensive_neighborhoods,
       most_expensive_neighborhoods_rent_sales,
       neighborhood_map)

# Create a Title for the Dashboard
column = pn.Column('# Deep Dive on Upscale San Francisco Real Estate', tabs )


# Create a tab layout for the dashboard
tabs = pn.Tabs(('Housing Units Per Year', housing_units_per_year),
              ('Average Gross Rent in San Francisco Per Year', average_gross_rent),
              ('Average Sales Price Per Year', average_sales_price),
              ('Average Prices by Neighborhood', average_price_by_neighborhood),
              ('Top 10 Most Expensive Neighborhoods', top_most_expensive_neighborhoods),
              ('Comparison of Rent and Sales', most_expensive_neighborhoods_rent_sales),
              ('Neighborhood Map', neighborhood_map))



In [73]:
## Serve the Panel Dashboard
column.servable()

Column
    [0] Markdown(str)
    [1] Tabs
        [0] Column
            [0] Column()
            [1] Row
                [0] HoloViews(Bars, name='interactive09670')
        [1] Column
            [0] Column()
            [1] Row
                [0] HoloViews(NdOverlay, name='interactive09796')
        [2] Column
            [0] Column()
            [1] Row
                [0] HoloViews(Curve, name='interactive09901')
        [3] Column
            [0] Column()
            [1] Row
                [0] Row(name='interactive09983')
                    [0] HoloViews(DynamicMap, name='interactive09983')
                    [1] Column
                        [0] WidgetBox
                            [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                        [1] VSpacer()
        [4] Column
            [0] Column()
            [1] Row
                [0] HoloViews(Bars, name='interactive10020')
        [5] Column
            [0] Column()
            [1] Row
                [0] Row(name='interactive10104')
                    [0] HoloViews(DynamicMap, name='interactive10104')
                    [1] Column
                        [0] WidgetBox
                            [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                        [1] VSpacer()
        [6] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive10140', relayout_data={'mapbox.center': {'lon': ...})

In [77]:
# Serve the# dashboard
!panel serve dashboard.ipynb --log-level debug --show

2021-03-29 02:17:40,805 Starting Bokeh server version 2.2.3 (running on Tornado 6.0.4)
2021-03-29 02:17:40,805 Cannot start Bokeh server, port 5006 is already in use
